In [41]:
import pickle
import numpy as np


In [42]:
with open('/Users/dhirajpoddar/Documents/Studies/project_solar/nlp_chat_bot/app/data/train_qa.txt', 'rb') as f:
    train_data = pickle.load(f)

In [43]:
with open('/Users/dhirajpoddar/Documents/Studies/project_solar/nlp_chat_bot/app/data/train_qa.txt','rb') as f:
    test_data = pickle.load(f)

In [44]:
type(test_data)

list

In [45]:
type(train_data)

list

In [46]:
len(train_data)

10000

train data consist of story, question and answer.

In [47]:
# First item of the training data, this is a story.
' '.join(train_data[0][0])

'Mary moved to the bathroom . Sandra journeyed to the bedroom .'

In [48]:
# First item of the training data, this is a question.

' '.join(train_data[0][1])

'Is Sandra in the hallway ?'

In [49]:
# First item of the training data, this is an answer.

train_data[0][2]

'no'

In [50]:
all_data = train_data + test_data 

In [51]:
len(all_data)

20000

In [117]:
test_data[0]




(['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'Sandra',
  'journeyed',
  'to',
  'the',
  'bedroom',
  '.'],
 ['Is', 'Sandra', 'in', 'the', 'hallway', '?'],
 'no')

In [52]:
vocab = set()
for story, question, answer in all_data:
    vocab = vocab.union(set(story))
    vocab = vocab.union(set(question))

In [53]:
vocab.add('no')
vocab.add('yes')

In [54]:
vocab_len = len(vocab) + 1

In [55]:
vocab_len

38

In [56]:
# longest story, question

max_story_len = max(len(data[0]) for data in all_data)
max_question_len = max(len(data[1]) for data in all_data)

In [57]:
print(f'maximun length of story : {max_story_len} \nmaximun length of question : {max_question_len}')

maximun length of story : 156 
maximun length of question : 6


In [58]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [59]:
tokenizer = Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)

In [60]:
train_story_text = []
train_question_text = []
train_answer = []

In [61]:
for story, question, answer in train_data:
    train_story_text.append(story)
    train_question_text.append(question)
    train_answer.append(answer)

In [62]:
train_story_seq = tokenizer.texts_to_sequences(train_story_text)

In [63]:
len(train_story_seq)

10000

In [64]:
def vectorize_stories(data, word_index=tokenizer.word_index, max_story_len=max_story_len, max_question_len=max_question_len):
    # train stories
    X = []

    #questions
    Xq = []

    # answer
    Y = []

    for story, question, answer in data:
        x = [word_index[word.lower()] for word in story]
        xq = [word_index[word.lower()] for word in question]
        y = np.zeros(len(word_index) + 1)
        y[word_index[answer]] = 1

        X.append(x)
        Xq.append(xq)
        Y.append(y)

    return (pad_sequences(X, maxlen=max_story_len), pad_sequences(Xq,maxlen=max_question_len), np.array(Y))

In [65]:
story_train, question_train, answers_train = vectorize_stories(train_data)


In [66]:
story_test, question_test, answers_test = vectorize_stories(test_data)

In [67]:
answers_train.shape

(10000, 38)

In [68]:
from keras.models import Sequential, Model
from keras.layers import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot, concatenate, LSTM

In [69]:
input_sequence = Input((max_story_len,))
question = Input((max_question_len,))

In [70]:
vocab_size = len(vocab) + 1

In [71]:
# INPUT ENCODER M
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size, output_dim=64))
input_encoder_m.add(Dropout(0.3))

#(samples, story_maxlen, embedding_dim)

In [72]:
# INPUT ENCODER C
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size, output_dim=max_question_len))
input_encoder_c.add(Dropout(0.3))

#OUTPUT
#(samples, story_maxlen, embedding_dim)

In [73]:
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size, output_dim=64, input_length=max_question_len))
question_encoder.add(Dropout(0.3))

#(Samples, query_maxlen, embedding_dim)

In [74]:
# ENCODED <---- ENCODER(INPUT)

input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

In [75]:
match = dot([input_encoded_m,question_encoded], axes=(2,2))
match = Activation('softmax')(match)

In [76]:
response = add([match, input_encoded_c])
response = Permute((2,1))(response)

In [77]:
answer = concatenate([response,question_encoded])

In [78]:
answer

<KerasTensor: shape=(None, 6, 220) dtype=float32 (created by layer 'concatenate')>

In [79]:
answer = LSTM(32)(answer)

In [80]:
answer=Dropout(0.5)(answer)
answer = Dense(vocab_size)(answer) # (samples, vocab_size) # YES/NO 000

In [81]:
answer = Activation('softmax')(answer)

In [82]:
model = Model([input_sequence, question], answer)

In [83]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [84]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 156)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 6)]          0           []                               
                                                                                                  
 sequential (Sequential)        (None, None, 64)     2432        ['input_1[0][0]']                
                                                                                                  
 sequential_2 (Sequential)      (None, 6, 64)        2432        ['input_2[0][0]']                
                                                                                              

In [85]:
# history = model.fit([story_train, question_train],answers_train, batch_size=32, epochs=100, validation_data=([story_test,question_test], answers_test))

In [86]:
model.save('model_bot_100_epoch.h5')

In [87]:
import matplotlib.pyplot as plt

In [88]:
def plot_loss():
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train','test'], loc='upper left')
    plt.show()


In [89]:
# plot_loss()

In [90]:
pred_results = model.predict([story_test, question_test])

313/313 [==============================] - 2s 5ms/step


In [91]:
pred_results

array([[0.02789798, 0.02831307, 0.02575055, ..., 0.03661539, 0.02525011,
        0.03264128],
       [0.02770156, 0.02854307, 0.02590186, ..., 0.03677281, 0.02558688,
        0.03261283],
       [0.02784426, 0.0284947 , 0.02578772, ..., 0.03685833, 0.0254395 ,
        0.03266602],
       ...,
       [0.02777037, 0.02845954, 0.02585839, ..., 0.03678823, 0.02516274,
        0.03257582],
       [0.02789584, 0.0284175 , 0.02609248, ..., 0.03669132, 0.02552887,
        0.03275908],
       [0.02798067, 0.02828041, 0.02595262, ..., 0.03657052, 0.02560394,
        0.03260307]], dtype=float32)

In [92]:
pred_results.shape

(10000, 38)

In [93]:
pred_results[0]

array([0.02789798, 0.02831307, 0.02575055, 0.032359  , 0.02447751,
       0.02422727, 0.02547853, 0.02427454, 0.02198295, 0.02467345,
       0.02374726, 0.02231968, 0.02889396, 0.02903337, 0.02844104,
       0.02344917, 0.02594864, 0.02256044, 0.02476883, 0.0278773 ,
       0.02671368, 0.02397184, 0.02286775, 0.02445862, 0.02337895,
       0.0270092 , 0.02424921, 0.02747937, 0.02515253, 0.02811662,
       0.02525835, 0.0304633 , 0.02469651, 0.02718099, 0.02802178,
       0.03661539, 0.02525011, 0.03264128], dtype=float32)

In [94]:
val_max = np.argmax(pred_results[0])

In [95]:
tokenizer.word_index.items()

dict_items([('to', 1), ('sandra', 2), ('left', 3), ('hallway', 4), ('travelled', 5), ('mary', 6), ('went', 7), ('football', 8), ('john', 9), ('journeyed', 10), ('up', 11), ('no', 12), ('dropped', 13), ('in', 14), ('bathroom', 15), ('apple', 16), ('took', 17), ('the', 18), ('daniel', 19), ('put', 20), ('milk', 21), ('garden', 22), ('there', 23), ('office', 24), ('kitchen', 25), ('grabbed', 26), ('back', 27), ('.', 28), ('yes', 29), ('down', 30), ('moved', 31), ('got', 32), ('?', 33), ('bedroom', 34), ('discarded', 35), ('picked', 36), ('is', 37)])

In [96]:
for key,val in tokenizer.word_index.items():
    if val == val_max:
        k = key

In [97]:
k

'discarded'

In [98]:
pred_results[0][val_max]

0.036615387

In [99]:
import tensorflow

In [101]:
my_model = model
my_model.load_weights('/Users/dhirajpoddar/Documents/Studies/project_solar/nlp_chat_bot/app/saved_model/model_bot_100_epoch.h5')

In [104]:
pred_result = my_model.predict([story_test, question_test])

313/313 [==============================] - 1s 4ms/step


In [111]:
pred_result[0]

array([7.3679075e-12, 6.4285357e-12, 5.4846865e-12, 6.1949404e-12,
       6.4994273e-12, 4.1704769e-12, 4.8531613e-12, 6.0450204e-12,
       8.4006231e-12, 5.2262630e-12, 6.3198964e-12, 7.0841067e-12,
       5.4211401e-12, 5.1875704e-12, 7.2293276e-01, 6.8731223e-12,
       5.7895598e-12, 5.5616518e-12, 5.0067589e-12, 6.9209534e-12,
       5.7617990e-12, 5.3991403e-12, 6.4727555e-12, 5.9921694e-12,
       5.4638048e-12, 8.0410132e-12, 5.3062368e-12, 4.4422283e-12,
       7.4817080e-12, 6.7860548e-12, 6.1544264e-12, 4.2367347e-12,
       5.1033665e-12, 6.5678929e-12, 2.7706724e-01, 6.7338110e-12,
       5.4199206e-12, 5.0851966e-12], dtype=float32)

In [105]:
val_max = np.argmax(pred_result[0])

In [108]:
for key, val in tokenizer.word_index.items():
    if val == val_max:
        k = key

In [109]:
k

'in'

In [113]:
w = '.'
print(w.lower())

.
